In [258]:
import pandas as pd
import re
import numpy as np


In [259]:
df_half = pd.read_csv('final_Final.csv')

In [260]:
df_half.head()

,Unnamed: 0,Links,Game Names,Price,OS,Processor,Memory,Graphics,DirectX,Storage,month,day,years
0,0,https://store.steampowered.com/app/816340/King...,Kingdom Rush Origins Tower Defense,Your Price:248.95 SR,*: Windows 7,Dual Core CPU,1 GB RAM,OpenGL 3.0 compliant with 512MB of video RAM.,NaN,500 MB available space,Oct,18,2018
1,1,https://store.steampowered.com/app/34270/SEGA_...,SEGA Mega Drive and Genesis Classics,No Price,*: Windows® XP or higher,NaN,NaN,32MB or greater,Version 9.0 VR Support: SteamVR or Oculus PC. ...,NaN,Release,No,No Release Date
2,2,https://store.steampowered.com/app/2407261/War...,War Thunder Su39 Pack,No Price,"*: Windows XP SP2, Windows Vista SP1, Windows 7","2,2 GHz",1 GB RAM,Nvidia GeForce 6XXX series or higher; AMD Rade...,®: 9.0c Hard Drive: 8 GB HD space Other Requir...,NaN,Jun,14,2023
3,3,https://store.steampowered.com/app/874390/The_...,The Battle of Polytopia,79.04 SR,*: Windows XP SP2 or later,Dual Core 2.0 Ghz,1 GB RAM,128mb Video,Version 9.0,160 MB available space,Aug,4,2020
4,4,https://store.steampowered.com/app/2506610/The...,The Riftbreaker Heart of the Swamp,Your Price:106.00 SR,*: Windows 8.1,and operating system,8 GB RAM,Nvidia GTX 750 2GB or AMD R7 265 2GB,Version 11,12 GB available space Sound Card:,Jun,21,2024


In [261]:
df_half.duplicated().sum()

0

In [262]:
# null values in:
# Processor 597
# Memory 721
# Graphics 1218 
# OS 378
# DirectX  1627
# Storage 939
df_half.isnull().sum()

Unnamed: 0       0
Links            0
Game Names       0
Price            0
OS             378
Processor      597
Memory         721
Graphics      1218
DirectX       1627
Storage        939
month            0
day              0
years            0
dtype: int64

# Drop Unnecessary columns

In [263]:
# OS is not in our feature target for clustering so we will drop it
# 
df_steam = df_half.drop(['Unnamed: 0', 'Links', 'OS', 'DirectX', 'month', 'day'], axis=1)
df_steam

,Game Names,Price,Processor,Memory,Graphics,Storage,years
0,Kingdom Rush Origins Tower Defense,Your Price:248.95 SR,Dual Core CPU,1 GB RAM,OpenGL 3.0 compliant with 512MB of video RAM.,500 MB available space,2018
1,SEGA Mega Drive and Genesis Classics,No Price,NaN,NaN,32MB or greater,NaN,No Release Date
2,War Thunder Su39 Pack,No Price,"2,2 GHz",1 GB RAM,Nvidia GeForce 6XXX series or higher; AMD Rade...,NaN,2023
3,The Battle of Polytopia,79.04 SR,Dual Core 2.0 Ghz,1 GB RAM,128mb Video,160 MB available space,2020
4,The Riftbreaker Heart of the Swamp,Your Price:106.00 SR,and operating system,8 GB RAM,Nvidia GTX 750 2GB or AMD R7 265 2GB,12 GB available space Sound Card:,2024
...,...,...,...,...,...,...,...
8972,Siege Survival Gloria Victis,118.80 SR,Intel Core i3 or AMD Athlon II X4 with at leas...,4 GB RAM,"GeForce GT 740, Radeon HD 7750 - at least 1GB ...",10 GB,2021
8973,Lay a Beauty to Rest A Butterfly Dream,34.10 SR,Intel Core i3 2.00 GHz 或 AMD A10,2 GB RAM,Intel® HD Graphics 3000,3 GB,2022
8974,Pamali Indonesian Folklore Horror,34.64 SR,Intel(R) Core(TM) i5-7200U CPU @ 2.50Ghz,4 GB RAM,NVIDIA GeForce 940MX,2 GB,2018
8975,A fascinating story,56.04 SR,Intel Core i5,4 GB RAM,NVIDIA Gef750,8 GB,2024


# Dealing with missing and incorrect values

> ## 'n' value in Storage feature:

In [264]:
# dealing with the value 'n' in the storage 
# we have 337 observations with 'n' value in the storage
df_steam[df_steam['Storage'] == 'n'].value_counts().sum() # it gives 279

279

In [265]:
df_steam[df_steam['Storage'] == 'n'] # it gives 383

,Game Names,Price,Processor,Memory,Graphics,Storage,years
4798,Revue Starlight El Dorado,No Price,Intel Core 2 DUO 2.0GHz 相当以上,2 GB RAM,VRAM 320MB,n,2024
4805,The Jackbox Party Starter,No Price,2.66 Ghz Core 2 Duo or Greater,4 GB RAM,GeForce 500+ / Radeon 5000+ or Greater,n,2022
4834,THE KING OF FIGHTERS XV DLC Team Pass Team Pa...,35.70 SR,Intel Core i5,8 GB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,n,2022
4852,Age of Empires II Retired,No Price,1.2GHZ CPU,1 GB RAM,Direct X 9.0c Capable GPU,n,2013
4907,The Survivalists,8.99 SR,Intel Core 2 Duo E8400,2 GB RAM,DirectX 11 Graphics Adapter with 1 GB VRAM,n,2020
...,...,...,...,...,...,...,...
8910,WRC 9 Career Starter Upgrades,4.77 SR,Intel Core i3-2100,4 GB RAM,"Nvidia GeForce GTX 650, 1 Go",n,2021
8918,DRAGON BALL XENOVERSE 2 Extra DLC Pack 3,No Price,Intel Core i5-3570 / AMD Ryzen 5 1600,4 GB RAM,Nvidia GeForce GTX 1030 / AMD Radeon HD 7770,n,2018
8953,DRAGON BALL FighterZ Bardock,10.00 SR,"AMD FX-4350, 4.2 GHz / Intel Core i5-3470, 3.2...",4 GB RAM,"Radeon HD 6870, 1 GB / GeForce GTX 650 Ti, 1 GB",n,2018
8954,Armello The Dragon Clan,179.93 SR,Dual-core 2.0 GHz,2 GB RAM,DirectX 10 class GPU with 512MB VRAM,n,2019


In [266]:
df_steam['Memory'].unique()

array(['1 GB RAM', nan, '8 GB RAM', '500 MB RAM', '3 GB RAM',
       '1 GB RAM Hard Disk Space: 2GB Video Card:', '4 GB RAM',
       '2 GB RAM', '5 GB RAM', '8192 MB RAM', '10 GB RAM',
       '2GB or more Hard Drive: At least 7GB of free space', '512 MB RAM',
       '512 MB RAM Hard Disk Space: 8GB Video Card: 256 MB Video',
       '6 GB RAM', '2048 MB RAM', '16 GB RAM', '256 MB RAM',
       '2 GB Available System', '1 GB', '200 MB RAM', '1024 MB RAM',
       '1000 MB RAM', '12 GB RAM',
       '•  DX10 Video card with 512MB of VRAM • (NOTE: Intel HD Graphic 4000 NOT CURRENTLY SUPPORTED on Windows 8) •  2GB HDD space * Starting January 1st, 2024, the Steam Client will only support Windows 10 and later versions. Minimum: •',
       '2GB Hard Disk Space: 32GB for high resolution* (Allow for additional +3GB for international builds) Video Card: GeForce 7XXX / ATI Radeon HD 3000 series with 512MB or more of dedicated video',
       '2.5 GB RAM', '2GB RAM', '2 GB for 32-bit', '1 MB RAM',
   

In [267]:
# dealing with the value 'n' in the storage: cleaning
# we will check each unique ram value that has 'n' value in storage and then fill it with the mode of the memory


In [268]:
# 1: check 8 GB RAM
# there are 64 rows with memory 8 GB RAM and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '8 GB RAM')].value_counts().sum()

# now we will calculate the mode of 'Storage' where 'Memory' is '8 GB RAM' and 'Storage' is not 'n'
mode_storage_8gb = df_steam[(df_steam['Memory'] == '8 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

# impute the 'Storage' feature where 'Memory' is '8 GB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '8 GB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_8gb

df_steam[(df_steam['Memory'] == '8 GB RAM') & (df_steam['Storage'] == mode_storage_8gb)]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
4834,THE KING OF FIGHTERS XV DLC Team Pass Team Pa...,35.70 SR,Intel Core i5,8 GB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,10 GB,2022
4846,SKYBOX VR Video Player,No Price,"Intel i3-6100 / AMD Ryzen 3 1200, FX4350 or gr...",8 GB RAM,NVIDIA GTX 960 / AMD Radeon R9 290 or greater,10 GB,2017
4864,Dealer Simulator,20.11 SR,İntel Core i5 3.0 GHz,8 GB RAM,Nvidia Geforce GTX 1050ti 4 GB,10 GB,2024
4937,Ace Attorney Investigations Collection,No Price,Intel® Core™ Core i3 8350k or AMD Ryzen3 3200G,8 GB RAM,Intel® UHD Graphics 630 or Radeon™ Vega 8 Grap...,10 GB,2024
4940,DAVE THE DIVER,53.50 SR,Intel Core i3 Dual Core,8 GB RAM,NVIDIA Geforce GTS 450 / AMD Radeon HD 5570,10 GB,2023
...,...,...,...,...,...,...,...
8766,Phoenix Point Corrupted Horizons DLC,56.64 SR,Intel Core i3 / AMD Phenom II X4,8 GB RAM,Nvidia GeForce GTX 950 / AMD Radeon R9 270x,10 GB,2021
8792,SILT,No Price,Intel Core i5 - 8300H 2.3Ghz,8 GB RAM,Nvidia GTX 1050 - 4GB,10 GB,2022
8871,Arcadegeddon,55.44 SR,"Intel® CPU with at least 4 threads, 2.3GHz or ...",8 GB RAM,NVIDIA® GeForce® GTX 670 GPU and/or AMD Radeon...,10 GB,2023
8934,Disco Simulator,37.51 SR,Intel Core i3 3.0 GHz,8 GB RAM,NVidia GeForce GTX 970,10 GB,2024


In [269]:
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '8 GB RAM')]

,Game Names,Price,Processor,Memory,Graphics,Storage,years


In [270]:
# 2: check 4 GB RAM
# there are 77 rows with memory 4 GB and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '4 GB RAM')].value_counts().sum()

# now we will calculate the mode of 'Storage' where 'Memory' is '4 GB RAM' and 'Storage' is not 'n'
mode_storage_4gb = df_steam[(df_steam['Memory'] == '4 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

# impute the 'Storage' feature where 'Memory' is '4 GB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '4 GB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_4gb

df_steam[(df_steam['Memory'] == '4 GB RAM') & (df_steam['Storage'] == mode_storage_4gb)]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
4805,The Jackbox Party Starter,No Price,2.66 Ghz Core 2 Duo or Greater,4 GB RAM,GeForce 500+ / Radeon 5000+ or Greater,2 GB,2022
4843,Monster Prom 3 Monster Roadtrip,79.90 SR,Anything from the last 5 years,4 GB RAM,NaN,2 GB,2022
4873,Figment 2 Creed Valley,7.72 SR,Intel Core i5 2500 / AMD FX 6120,4 GB RAM,Nvidia GTX 650 Ti / Nvidia GT 750M / Radeon HD...,2 GB,2023
4883,Gatekeeper,49.89 SR,Intel Core i3-6100,4 GB RAM,Geforce GTX 750,2 GB,2024
4929,Expansion Hearts of Iron IV Arms Against Tyranny,48.00 SR,Intel® Core™ i5 750,4 GB RAM,Nvidia® GeForce™ GTX 470 (1.28GB),2 GB,2023
...,...,...,...,...,...,...,...
8901,HalluciSabbat of Koishi,29.79 SR,Any CPU,4 GB RAM,Any,2 GB,2022
8910,WRC 9 Career Starter Upgrades,4.77 SR,Intel Core i3-2100,4 GB RAM,"Nvidia GeForce GTX 650, 1 Go",2 GB,2021
8918,DRAGON BALL XENOVERSE 2 Extra DLC Pack 3,No Price,Intel Core i5-3570 / AMD Ryzen 5 1600,4 GB RAM,Nvidia GeForce GTX 1030 / AMD Radeon HD 7770,2 GB,2018
8953,DRAGON BALL FighterZ Bardock,10.00 SR,"AMD FX-4350, 4.2 GHz / Intel Core i5-3470, 3.2...",4 GB RAM,"Radeon HD 6870, 1 GB / GeForce GTX 650 Ti, 1 GB",2 GB,2018


In [271]:
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '4 GB RAM')]

,Game Names,Price,Processor,Memory,Graphics,Storage,years


In [272]:
# 3: check 6 GB RAM
# there are 12 rows with memory 6 GB RAM and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '6 GB RAM')].value_counts().sum()

# now we will calculate the mode of 'Storage' where 'Memory' is '6 GB RAM' and 'Storage' is not 'n'
mode_storage_6gb = df_steam[(df_steam['Memory'] == '6 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

# impute the 'Storage' feature where 'Memory' is '6 GB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '6 GB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_6gb

df_steam[(df_steam['Memory'] == '6 GB RAM') & (df_steam['Storage'] == mode_storage_6gb)]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
246,Crusader Kings III Roads to Power,Your Price:132.80 SR,and operating system,6 GB RAM,Nvidia® GeForce™ GTX 660 (2GB) / AMD® Radeon™ ...,8 GB available space,2024
673,Crusader Kings III Content Creator Pack North ...,No Price,and operating system,6 GB RAM,Nvidia® GeForce™ GTX 660 (2GB) / AMD® Radeon™ ...,8 GB available space,2024
925,Shieldwall,Your Price:80.00 SR,and operating system,6 GB RAM,NVIDIA GeForce GTX 670 or AMD R9 270,8 GB available space,2023
1203,Crusader Kings III,Your Price:132.80 SR,and operating system,6 GB RAM,Nvidia® GeForce™ GTX 660 (2GB) / AMD® Radeon™ ...,8 GB available space,2020
1306,Crusader Kings III Legends of the Dead,Your Price:132.80 SR,and operating system,6 GB RAM,Nvidia® GeForce™ GTX 660 (2GB) / AMD® Radeon™ ...,8 GB available space,2024
1317,Crusader Kings III Tours Tournaments,Your Price:431.10 SR,and operating system,6 GB RAM,Nvidia® GeForce™ GTX 660 (2GB) / AMD® Radeon™ ...,8 GB available space,2023
1381,Dead Frontier 2,No Price,and operating system,6 GB RAM,Radeon HD 8550 or equivalent,8 GB available space,2019
1389,Crusader Kings III Royal Court,Your Price:431.10 SR,and operating system,6 GB RAM,Nvidia® GeForce™ GTX 660 (2GB) / AMD® Radeon™ ...,8 GB available space,2022
1795,Executive Assault 2,No Price,and operating system,6 GB RAM,"Mid range - 4GB VRAM, game is graphically inte...",8 GB available space,2023
2086,Crusader Kings III Royal Court,Your Price:431.10 SR,and operating system,6 GB RAM,Nvidia® GeForce™ GTX 660 (2GB) / AMD® Radeon™ ...,8 GB available space,2022


In [273]:
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '6 GB RAM')]

,Game Names,Price,Processor,Memory,Graphics,Storage,years


In [274]:
# checking 2 GB
#df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '2 GB RAM') | (df_steam['Memory'] == '2GB RAM') | (df_steam['Memory'] == '2 GB')]


In [275]:
# 4.1: check 2 GB RAM (there are more than value for 2 GB)

# there are 39 and more i think rows with memory 2 GB RAM and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '2 GB RAM')].value_counts().sum()

# now we will calculate the mode of 'Storage' where 'Memory' is '2 GB RAM' and 'Storage' is not 'n'
mode_storage_2gb_RAM = df_steam[(df_steam['Memory'] == '2 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

# impute the 'Storage' feature where 'Memory' is '2 GB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '2 GB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_2gb_RAM

df_steam[(df_steam['Memory'] == '2 GB RAM') & (df_steam['Storage'] == mode_storage_2gb_RAM)]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
4798,Revue Starlight El Dorado,No Price,Intel Core 2 DUO 2.0GHz 相当以上,2 GB RAM,VRAM 320MB,2 GB,2024
4837,DUSK,120.04 SR,2.4GHZ Dual Core Processor Or Higher,2 GB RAM,GeForce 9800GT Or Equivalent,2 GB,2018
4866,Aero GPX,No Price,Celeron G5905,2 GB RAM,Intel® UHD Graphics 610,2 GB,2024
4885,ClueCluedo Classic Edition,80.98 SR,2 GHz single core processor,2 GB RAM,DirectX 10 class GPU [1280 x 720],2 GB,2018
4907,The Survivalists,8.99 SR,Intel Core 2 Duo E8400,2 GB RAM,DirectX 11 Graphics Adapter with 1 GB VRAM,2 GB,2020
...,...,...,...,...,...,...,...
8777,Bean Battles,1.37 SR,1.5GHZ,2 GB RAM,512 MB VRAM Intel HD 4000 / GeForce 200 Series...,2 GB,2018
8782,FPV Freerider Recharged,No Price,Intel Core2 Duo 2Ghz or Higher/ AMD 2Ghz or Hi...,2 GB RAM,"Geforce GTX 960M / Radeon HD 7750 or higher, 1...",2 GB,2016
8835,Gensokyo Defenders,133.00 SR,intelCorei7 or newer,2 GB RAM,GeForceGTX550 or newer,2 GB,2019
8893,Trackmania2 Lagoon,579.65 SR,Dual core from Intel or AMD at 2 GHz or better,2 GB RAM,Video card must be 512 MB or more and should b...,2 GB,2017


In [276]:
# 4.2: Check 2 GB (there are rows with memory '2 GB' that have 'n' value in storage)
num_rows_2gb_n_storage = df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '2 GB')].value_counts().sum()
#print(f"Number of rows with '2 GB' memory and 'n' in storage: {num_rows_2gb_n_storage}")

# Calculate the mode of 'Storage' where 'Memory' is '2 GB RAM' and 'Storage' is not 'n'
# we have only two rows for '2 GB' that's why we imput it with '2 GB RAM'
mode_storage_2gb_ram = df_steam[(df_steam['Memory'] == '2 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()

# Check if the mode calculation returned a non-empty result
if not mode_storage_2gb_ram.empty:
    mode_storage_2gb_ram_value = mode_storage_2gb_ram.iloc[0]
    # Impute 'Storage' where 'Memory' is '2 GB' and 'Storage' is 'n'
    df_steam.loc[(df_steam['Memory'] == '2 GB') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_2gb_ram_value
    
df_steam[(df_steam['Memory'] == '2 GB') & (df_steam['Storage'] == 'n')]

,Game Names,Price,Processor,Memory,Graphics,Storage,years


In [277]:
# 4.3: Check 2GB RAM (there are rows with memory '2 GB' that have 'n' value in storage)
num_rows_2gb_ = df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '2GB RAM')].value_counts().sum()
#print(f"Number of rows with '2 GB' memory and 'n' in storage: {num_rows_2gb_n_storage}")

# Calculate the mode of 'Storage' where 'Memory' is '2 GB RAM' and 'Storage' is not 'n'
# we have only minimum rows for '2GB RAM' that's why we imput it with '2 GB RAM'
mode_storage_2gbRAM = df_steam[(df_steam['Memory'] == '2 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()

# Check if the mode calculation returned a non-empty result
if not mode_storage_2gbRAM.empty:
    mode_storage_2gbRAM_value = mode_storage_2gbRAM.iloc[0]
    # Impute 'Storage' where 'Memory' is '2 GB' and 'Storage' is 'n'
    df_steam.loc[(df_steam['Memory'] == '2GB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_2gbRAM_value
    
df_steam[(df_steam['Memory'] == '2GB RAM') & (df_steam['Storage'] == 'n')]

,Game Names,Price,Processor,Memory,Graphics,Storage,years


In [278]:
df_steam[(df_steam['Memory'] == '12 GB RAM') & (df_steam['Storage'] == 'n')]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
4953,HalfLife Alyx,No Price,Core i5-7500 / Ryzen 5 1600,12 GB RAM,GTX 1060 / RX 580 - 6GB VRAM,n,2020
4985,THE FINALS Metro Drifter Set,No Price,Intel Core i5-6600K or AMD Ryzen R5 1600 proce...,12 GB RAM,NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 580,n,2024
5942,THE FINALS,No Price,Intel Core i5-6600K or AMD Ryzen R5 1600 proce...,12 GB RAM,NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 580,n,2023
6088,THE FINALS Season 3 Starter Pack,No Price,Intel Core i5-6600K or AMD Ryzen R5 1600 proce...,12 GB RAM,NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 580,n,2024
7395,THE FINALS Bank Rabbit Set,No Price,Intel Core i5-6600K or AMD Ryzen R5 1600 proce...,12 GB RAM,NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 580,n,2024


In [279]:
# 5: check 12 GB RAM
# there are 5 rows with memory 12 GB RAM and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '12 GB RAM')].value_counts().sum()

# now we will calculate the mode of 'Storage' where 'Memory' is '12 GB RAM' and 'Storage' is not 'n'
mode_storage_12gb = df_steam[(df_steam['Memory'] == '12 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

# impute the 'Storage' feature where 'Memory' is '12 GB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '12 GB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_12gb

df_steam[(df_steam['Memory'] == '12 GB RAM') & (df_steam['Storage'] == mode_storage_12gb)]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
4825,Hell Let Loose Operation Luttich Units,No Price,Intel Core i5-6600/AMD Ryzen 3 1300X,12 GB RAM,NVIDIA GeForce GTX 960 4GB/AMD Radeon R9 380 4GB,30 GB,2024
4953,HalfLife Alyx,No Price,Core i5-7500 / Ryzen 5 1600,12 GB RAM,GTX 1060 / RX 580 - 6GB VRAM,30 GB,2020
4985,THE FINALS Metro Drifter Set,No Price,Intel Core i5-6600K or AMD Ryzen R5 1600 proce...,12 GB RAM,NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 580,30 GB,2024
5248,Park Beyond,No Price,Intel® Core™ i7-3770 / AMD Ryzen™ 5 1400,12 GB RAM,1080p/30fps: NVIDIA GeForce GTX 660 2GB / AMD ...,30 GB,2023
5942,THE FINALS,No Price,Intel Core i5-6600K or AMD Ryzen R5 1600 proce...,12 GB RAM,NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 580,30 GB,2023
5985,Hell Let Loose,155.99 SR,Intel Core i5-6600/AMD Ryzen 3 1300X,12 GB RAM,NVIDIA GeForce GTX 960 4GB/AMD Radeon R9 380 4GB,30 GB,2021
6088,THE FINALS Season 3 Starter Pack,No Price,Intel Core i5-6600K or AMD Ryzen R5 1600 proce...,12 GB RAM,NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 580,30 GB,2024
6159,Hell Let Loose Oak Leaf,333.09 SR,Intel Core i5-6600/AMD Ryzen 3 1300X,12 GB RAM,NVIDIA GeForce GTX 960 4GB/AMD Radeon R9 380 4GB,30 GB,2024
6176,Hell Let Loose Polish Parachute Brigade,333.09 SR,Intel Core i5-6600/AMD Ryzen 3 1300X,12 GB RAM,NVIDIA GeForce GTX 960 4GB/AMD Radeon R9 380 4GB,30 GB,2024
6234,Hell Let Loose Pea Dot,333.09 SR,Intel Core i5-6600/AMD Ryzen 3 1300X,12 GB RAM,NVIDIA GeForce GTX 960 4GB/AMD Radeon R9 380 4GB,30 GB,2023


In [280]:
df_steam[(df_steam['Memory'] == '512 MB RAM') & (df_steam['Storage'] == 'n')]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
5070,Sid Meiers Civilization IV,8.40 SR,Intel chipset,512 MB RAM,NaN,n,2005
6307,Nancy Drew Ghost of Thornton Hall,94.25 SR,1.5 GHZ or greater Pentium 4 CPU or equivalent...,512 MB RAM,128 MB DirectX 9.0 compatible video card,n,2013
6341,Myst Masterpiece Edition,191.00 SR,1.8 GHz or greater,512 MB RAM,NaN,n,1999
7497,Kentucky Route Zero PC Edition,No Price,1 GHz,512 MB RAM,Directx 9.0c compatible video card,n,2013
7804,Hylics,No Price,2.0 GHz or faster,512 MB RAM,1024 x 768 or higher desktop resolution,n,2015
7813,Nancy Drew Ghost of Thornton Hall,94.25 SR,1.5 GHZ or greater Pentium 4 CPU or equivalent...,512 MB RAM,128 MB DirectX 9.0 compatible video card,n,2013
8097,Baldurs Gate Enhanced Edition Official Soundtrack,104.87 SR,1 GHZ,512 MB RAM,OpenGL 2.0 compatible,n,2016
8099,Baldurs Gate Siege of Dragonspear Official Sou...,104.87 SR,1 GHZ,512 MB RAM,OpenGL 2.0 compatible,n,2016
8721,RPG Maker VX Ace,No Price,Intel® Pentium® 4 2.0 GHz equivalent or faster...,512 MB RAM,1024 x 768 pixels or higher desktop resolution,n,2012
8864,Living Legends The Frozen Fear Collection,No Price,Pentium processor (or equivalent AMD® processo...,512 MB RAM,DirectX® compatible video card,n,2015


In [281]:
#df_steam[df_steam['Storage'] == 'n']

In [282]:
# 6: check 512 MB RAM
# there are 11 rows with memory 512 MB RAM and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '512 MB RAM')].value_counts().sum()

# now we will calculate the mode of 'Storage' where 'Memory' is '512 MB RAM' and 'Storage' is not 'n'
mode_storage_512gb = df_steam[(df_steam['Memory'] == '512 MB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

# impute the 'Storage' feature where 'Memory' is '512 MB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '512 MB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_512gb

df_steam[(df_steam['Memory'] == '512 MB RAM') & (df_steam['Storage'] == mode_storage_512gb)]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
480,RealFlight Trainer Edition,No Price,Intel® Pentium® 1.0GHz or equivalent,512 MB RAM,Full,2 GB available space,2020
1009,EMERGENCY 4 Deluxe,Your Price:61.48 SR,Single Core 1.7 GHz,512 MB RAM,NaN,2 GB available space,2006
1505,Wandersong,89.85 SR,Not that intense,512 MB RAM,Not that good,2 GB available space,2018
1949,Dungeons Dragons Dark Sun Series,13.17 SR,1.8 GHz,512 MB RAM,3D,2 GB available space,2022
3385,Forgotten Realms The Archives Collection One,13.17 SR,1.8 GHz,512 MB RAM,3D,2 GB available space,2022
3507,Forgotten Realms The Archives Collection Two,13.17 SR,1.8 GHz,512 MB RAM,3D,2 GB available space,2022
3792,Dungeons Dragons Krynn Series,13.17 SR,1.8 GHz,512 MB RAM,3D,2 GB available space,2022
3878,Dungeons Dragons Dark Sun Series,13.17 SR,1.8 GHz,512 MB RAM,3D,2 GB available space,2022
4137,Dungeons Dragons Ravenloft Series,13.17 SR,1.8 GHz,512 MB RAM,3D,2 GB available space,2022
5070,Sid Meiers Civilization IV,8.40 SR,Intel chipset,512 MB RAM,NaN,2 GB available space,2005


In [283]:
# 7: check 1 GB RAM
#there are 9 rows with memory 1 GB RAM and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '1 GB RAM')].value_counts().sum()

#now we will calculate the mode of 'Storage' where 'Memory' is '1 GB RAM' and 'Storage' is not 'n'
mode_storage_1gb = df_steam[(df_steam['Memory'] == '1 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

#impute the 'Storage' feature where 'Memory' is '1 GB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '1 GB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_1gb

df_steam[(df_steam['Memory'] == '1 GB RAM') & (df_steam['Storage'] == mode_storage_1gb)]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
4830,Strange Horticulture,177.05 SR,2.4 GHz,1 GB RAM,512 MB display memory,1 GB,2022
4852,Age of Empires II Retired,No Price,1.2GHZ CPU,1 GB RAM,Direct X 9.0c Capable GPU,1 GB,2013
5051,You Will Die Here Tonight,20.47 SR,2 GHz Intel Pentium 4 or AMD Athlon or equivalent,1 GB RAM,1 GB,1 GB,2023
5347,Sarahs Adventure Time Travel,No Price,intel core i3,1 GB RAM,NaN,1 GB,2024
5448,SONIC ADVENTURE 2 BATTLE,801.81 SR,Pentium 4 @ 3.2 GHz/Athlon 64 3000+ or Equival...,1 GB RAM,256 MB (NVIDIA GeForce 8600/AT Radeon HD3650) ...,1 GB,2012
5565,Quake Live,168.75 SR,2 GHz Intel Processor or better,1 GB RAM,"NVIDIA GeForce 4 MX or better, ATI Radeon 8500...",1 GB,2010
5621,Animaze by FaceRig,No Price,Intel® Core™ i5-4570 3.2 GHz or equivalent,1 GB RAM,NVIDIA GeForce GTX760 or equivalent (support f...,1 GB,2021
5678,Hero Zero Multiplayer RPG,No Price,Intel Pentium or compatible,1 GB RAM,NaN,1 GB,2016
5813,Head Basketball,No Price,2 Ghz,1 GB RAM,"128MB, OpenGL 3+",1 GB,2022
5969,Northgard,11.73 SR,Intel 2.0ghz Core 2 Duo or equivalent,1 GB RAM,Nvidia 450 GTS / Radeon HD 5750 or better,1 GB,2018


In [284]:
df_steam[df_steam['Storage'] == 'n']

,Game Names,Price,Processor,Memory,Graphics,Storage,years
5012,KOF XV Fighter Pass,59.50 SR,Intel Core i5,NaN,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,n,2023
5094,Q REMASTERED,10.25 SR,NaN,NaN,NaN,n,2023
5198,Endzone 2 Artbook,10.75 SR,NaN,NaN,NaN,n,2024
5200,Secret World Legends,No Price,2.6 GHZ Intel Core 2 DUO or equivalent AMD CPU,At least 2GB RAM for Windows XP / 3GB Ram for ...,NaN,n,2017
5342,KOF XV DLC Characters Team AWAKENED OROCHI,17.70 SR,Intel Core i5,8 MB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,n,2022
...,...,...,...,...,...,...,...
8786,Sakura Succubus 6,6.58 SR,1.2 GHz Pentium 4,NaN,1280 x 720,n,2022
8800,Automobilista 2 Monza Pack,No Price,3.5 GHz Intel Core i5 3450 or AMD FX-8350,5 GB RAM,GTX680 or equivalent,n,2021
8822,Clickteam Fusion 25,276.30 SR,200 Mhz Pentium processor or higher,256 MB RAM,Must support minimum of Direct3D 9,n,2013
8839,My Singing Monsters Playground,No Price,Intel Core i3 2.5GHz,NaN,GeForce GTX 660 or equivalent,n,2021


In [285]:
# 7: check 8 MB RAM
# there are some rows with memory 8 MB RAM and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '8 MB RAM')].value_counts().sum()

# now we will calculate the mode of 'Storage' where 'Memory' is '8 MB RAM' and 'Storage' is not 'n'
mode_storage_8mb = df_steam[(df_steam['Memory'] == '8 MB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

# impute the 'Storage' feature where 'Memory' is '8 MB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '8 MB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_8mb

df_steam[(df_steam['Memory'] == '8 MB RAM') & (df_steam['Storage'] == mode_storage_8mb)]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
5292,Farming Simulator 22 Pumps n Hoses Pack,No Price,Intel Core i5-3330 / AMD FX-8350 or equivalent,8 MB RAM,GeForce GTX 660 (2 GB) / Radeon R9 390X (8 GB),7 MB,2022
5342,KOF XV DLC Characters Team AWAKENED OROCHI,17.70 SR,Intel Core i5,8 MB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,7 MB,2022
5631,KOF XV DLC Characters Team SAMURAI,17.70 SR,Intel Core i5,8 MB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,7 MB,2022
6150,KOF XV DLC Character SHINGO YABUKI,11.50 SR,Intel Core i5,8 MB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,7 MB,2023
6497,KOF XV DLC Character NAJD,11.50 SR,Intel Core i5,8 MB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,7 MB,2023
7004,Farming Simulator 22 Platinum Expansion,100.49 SR,Intel Core i5-3330 / AMD FX-8350 or equivalent,8 MB RAM,GeForce GTX 660 (2 GB) / Radeon R9 390X (8 GB),7 MB,2022
7683,KOF XV DLC Character SYLVIE PAULA PAULA,11.50 SR,Intel Core i5,8 MB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,7 MB,2023
7765,KOF XV DLC Character KIM KAPHWAN,11.50 SR,Intel Core i5,8 MB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,7 MB,2023
7800,KOF XV DLC Character SHINGO YABUKI,11.50 SR,Intel Core i5,8 MB RAM,Nvidia GeForce GTX 770 2GB / AMD Radeon R9 280...,7 MB,2023
7999,FALLEN,34.64 SR,AMD Ryzen 3 1200 Intel Core i5-7500,8 MB RAM,AMD Radeon RX 560 with 4GB VRAM NVIDIA GeForce...,7 MB,2024


In [390]:
df_steam[df_steam['Storage'] == 'n']

,Game Names,Price,Processor,Memory,Graphics,Storage,years


In [287]:
# 8: check 16 GB RAM
# there are some rows with memory 16 GB RAM and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '16 GB RAM')].value_counts().sum()

# now we will calculate the mode of 'Storage' where 'Memory' is '16 GB RAM' and 'Storage' is not 'n'
mode_storage_16gb = df_steam[(df_steam['Memory'] == '16 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

# impute the 'Storage' feature where 'Memory' is '16 GB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '16 GB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_16gb

#df_steam[(df_steam['Memory'] == '16 GB RAM') & (df_steam['Storage'] == mode_storage_16gb)]

In [288]:
# 9: check 1GB System RAM
# there are some rows with memory 16 GB RAM and have 'n' value in storage
df_steam[(df_steam['Storage'] == 'n') & (df_steam['Memory'] == '16 GB RAM')].value_counts().sum()

# now we will calculate the mode of 'Storage' where 'Memory' is '16 GB RAM' and 'Storage' is not 'n'
mode_storage_16gb = df_steam[(df_steam['Memory'] == '16 GB RAM') & (df_steam['Storage'] != 'n')]['Storage'].mode()[0]

# impute the 'Storage' feature where 'Memory' is '16 GB RAM' and 'Storage' is 'n'
df_steam.loc[(df_steam['Memory'] == '16 GB RAM') & (df_steam['Storage'] == 'n'), 'Storage'] = mode_storage_16gb

#df_steam[(df_steam['Memory'] == '16 GB RAM') & (df_steam['Storage'] == mode_storage_16gb)]

In [289]:
#df_steam[df_steam['Storage'] == 'n']

In [290]:
# we will drop the rows when Storage is 'n' and Memory is null 
n_drop2 = (df_steam['Storage'] == 'n') & (df_steam['Memory'].isnull())

df_steam.drop(df_steam[n_drop2].index, inplace=True)

In [291]:
# we will drop the rows when Storage is 'n' and Graphics, Memory are nulls 
n_drop = (df_steam['Storage'] == 'n') & (df_steam['Graphics'].isnull()) & (df_steam['Memory'].isnull())

df_steam.drop(df_steam[n_drop].index, inplace=True)

In [292]:
df_steam[df_steam['Storage'] == 'n']

,Game Names,Price,Processor,Memory,Graphics,Storage,years
5200,Secret World Legends,No Price,2.6 GHZ Intel Core 2 DUO or equivalent AMD CPU,At least 2GB RAM for Windows XP / 3GB Ram for ...,NaN,n,2017
5369,Torchlight II,162.85 SR,x86-compatible 1.4GHz or faster processor,1GB System RAM,NaN,n,2012
5373,Dont Starve Hamlet,76.76 SR,1.7+ GHz or better,1+ gigs of RAM GB RAM,Radeon HD5450 or better; 256 MB or higher,n,2018
5414,Total War ROME II Greek States Culture Pack,319.50 SR,2 GHz Intel Dual Core processor / 2.6 GHz Inte...,"1GB RAM (XP), 2GB RAM (Vista / Windows 7 or 8)...",512 MB Direct 9.0c compatible card (shader mod...,n,2013
5427,Brothers in Arms Earned in Blood,No Price,1 GHz Pentium III or AMD Athlon (2.5 GHz Penti...,512 MB (1 GB recommended),32 MB Direct X 9.0c compliant (64 MB recommend...,n,2005
5470,DC Universe Online Power Bundle 2016,No Price,P4 3.0GHz or AMD equivalent,1GB RAM,NaN,n,2016
5523,Dont Starve Shipwrecked,76.76 SR,1.7+ GHz or better,1+ gigs of RAM GB RAM,Radeon HD5450 or better; 256 MB or higher,n,2015
5638,Dont Starve Shipwrecked,76.76 SR,1.7+ GHz or better,1+ gigs of RAM GB RAM,Radeon HD5450 or better; 256 MB or higher,n,2015
5740,Tales of Monkey Island Complete Season,29.50 SR,2.0 GHz + (3 GHz Pentium 4 or equivalent rec.),512MB (1GB rec.),64MB DirectX 8.1-compliant video card (128MB r...,n,2009
5783,Supreme Commander,106.40 SR,3.0 GHz Intel or equivalent AMD processor or b...,1 GB RAM or better,NaN,n,2007


In [293]:
# we need to drop the storage rows having value 'n' and unique memory because we cannot impute them based on other memory
df_steam[df_steam['Storage'] == 'n']['Memory'].unique()

array(['At least 2GB RAM for Windows XP / 3GB Ram for Windows Vista and Windows 7',
       '1GB System RAM', '1+ gigs of RAM GB RAM',
       '1GB RAM (XP), 2GB RAM (Vista / Windows 7 or 8) GB RAM',
       '512 MB (1 GB recommended)', '1GB RAM', '512MB (1GB rec.)',
       '1 GB RAM or better', '2 GB (4 GB recommended)',
       '1.5GB XP / 1.5GB Vista', '2GB - 16GB RAM', '2048 MB RAM',
       '256 MB of RAM', '6 MB RAM', '4GB', '512MB system RAM (1GB Vista)',
       '256 MB RAM', '340 MB MB RAM', '1984 MB RAM', '6 GB RAM (64-bit)',
       '512MB', '3 GB RAM', '4GB RAM', '4GB or higher',
       '1 GB RAM (XP), 2 GB RAM (Vista®/Windows® 7)', '8GB GB RAM',
       '1024 MB (XP)/2048 MB (VISTA/7)', '1GB of RAM', '512 MB or higher',
       '5 GB RAM', '100 MB RAM'], dtype=object)

In [294]:
# dropping the storage rows having value 'n' and unique memory
df_steam = df_steam[df_steam['Storage'] != 'n']
df_steam[df_steam['Storage'] =='n']   # no rows of value 'n'

,Game Names,Price,Processor,Memory,Graphics,Storage,years


> ## Dealing with null values of the features

> 1. Processor:  597 null values

In [295]:
df_steam['Processor'].isnull().sum()

559

In [296]:
# checking null values of processor with other features
df_steam[(df_steam['Processor'].isnull()) & (df_steam['Memory'].isnull()) & (df_steam['Storage'].isnull())]


,Game Names,Price,Processor,Memory,Graphics,Storage,years
1,SEGA Mega Drive and Genesis Classics,No Price,NaN,NaN,32MB or greater,NaN,No Release Date
8,SPORE Creepy Cute Parts Pack,Your Price:140.58 SR,NaN,NaN,NaN,NaN,2008
46,POSTAL 2 Paradise Lost,2.23 SR,NaN,NaN,NaN,NaN,2015
66,SPORE Galactic Adventures,Your Price:140.58 SR,NaN,NaN,NaN,NaN,2009
70,Call of Duty Black Ops Rezurrection Content Pack,No Price,NaN,NaN,NaN,NaN,No Release Date
...,...,...,...,...,...,...,...
4744,Beat Hazard,Your Price:98.12 SR,NaN,NaN,NaN,NaN,2010
4765,Flatout 3 Chaos Destruction,136.79 SR,NaN,NaN,NaN,NaN,2011
4771,Geneforge 1,241.28 SR,NaN,NaN,NaN,NaN,2001
4784,Disgaea 7 Vows of the Virtueless Cosmetic Set,Your Price:257.44 SR,NaN,NaN,NaN,NaN,2023


In [297]:
# dropping variable for rows where Processor, Memory, and Storage are all null
col_drop = (df_steam['Processor'].isnull()) & (df_steam['Memory'].isnull()) & (df_steam['Storage'].isnull())

# Drop
df_steam = df_steam.drop(df_steam[col_drop].index).reset_index(drop=True)

In [298]:
df_steam.isnull().sum()

Game Names      0
Price           0
Processor     235
Memory        313
Graphics      858
Storage       615
years           0
dtype: int64

In [299]:
# dropping the rows that have null values with the processor 
df_steam[(df_steam['Processor'].isnull()) & (df_steam['Memory'].isnull()) & (df_steam['Graphics'].isnull())]


,Game Names,Price,Processor,Memory,Graphics,Storage,years
356,SpaceEngine PRO,No Price,NaN,NaN,NaN,1 MB available space,2019
390,Hollow Knight Official Soundtrack,Your Price:62.32 SR,NaN,NaN,NaN,156 MB available space,2017
402,Sherlock Holmes Franchise Classic Soundtrack,9.50 SR,NaN,NaN,NaN,"300 MB available space * Starting February 15,...",2020
405,Sherlock Holmes Franchise Modern Soundtrack,9.50 SR,NaN,NaN,NaN,"350 MB available space * Starting February 15,...",2020
410,Frogwares Games Classic Soundtrack,9.50 SR,NaN,NaN,NaN,"260 MB available space * Starting February 15,...",2020
...,...,...,...,...,...,...,...
6766,SnezhinkaArtBook,No Price,NaN,NaN,NaN,1 GB,2024
7850,Darkest Dungeon Soundtrack,106.52 SR,NaN,NaN,NaN,182 MB,2016
8027,Typing Instructor for Kids Platinum 5,101.14 SR,NaN,NaN,NaN,480 MB,2010
8254,Encased RPG Soundtrack,3.80 SR,NaN,NaN,NaN,600 MB,2019


In [300]:
# dropping variable for rows where Processor, Memory, and Graphics are all null
col_drop2 = (df_steam['Processor'].isnull()) & (df_steam['Memory'].isnull()) & (df_steam['Graphics'].isnull())

df_steam.drop(df_steam[col_drop2].index, inplace=True)

In [301]:
# drop the rows that have null values with the processor for storage valu 'n'
df_steam[(df_steam['Processor'].isnull()) & (df_steam['Memory'].isnull()) & (df_steam['Storage'] == 'n')]


,Game Names,Price,Processor,Memory,Graphics,Storage,years


In [302]:
# dropping variable for rows where Processor, Memory are nulls, and storage of valu 'n'
col_drop3 = (df_steam['Processor'].isnull()) & (df_steam['Memory'].isnull()) & (df_steam['Storage'] == 'n')

df_steam.drop(df_steam[col_drop3].index, inplace=True)

In [303]:
# dropping the rows that have null values with the processor that have nulls with memory as well
df_steam[(df_steam['Processor'].isnull()) & (df_steam['Memory'].isnull())]


,Game Names,Price,Processor,Memory,Graphics,Storage,years
540,BZZZT,12.03 SR,NaN,NaN,"DX10, DX11, DX12 capable.","300 MB available space * Starting January 1st,...",2023
2390,Call to Arms Gates of Hell Talvisota,Your Price:165.97 SR,NaN,NaN,GTX 1050 / RX 550,10 GB available space Sound Card:,2022
2702,Darkest Dungeon The Crimson Court,Your Price:83.93 SR,NaN,NaN,Open GL 3.2+ Compliant,1 GB available space,2017
5929,RPG in a Box,No Price,NaN,NaN,OpenGL 3.3+,500 MB,2022
7668,VoiceAttack,No Price,NaN,NaN,800x600 minimum resolution,155 MB,2009
7872,Pinball FX Williams Pinball Twilight Zone,No Price,NaN,NaN,Nvidia GTX 960 or AMD R9 380,16 GB,2023
8374,Pinball FX Williams Pinball Twilight Zone,No Price,NaN,NaN,Nvidia GTX 960 or AMD R9 380,16 GB,2023


In [304]:
# dropping the rows that have null values with the processor that have nulls with memory as well
col_drop4 = (df_steam['Processor'].isnull()) & (df_steam['Memory'].isnull())

df_steam.drop(df_steam[col_drop4].index, inplace=True)

In [305]:
df_steam[df_steam['Processor'].isnull()]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
91,Warhammer The Horus Heresy Legions,No Price,NaN,1 GB RAM,NaN,500 MB available space,2019
192,Call to Arms Gates of Hell Ostfront,Your Price:165.97 SR,NaN,8 GB RAM,GTX 1050 / RX 550,70 GB available space Sound Card:,2021
204,Rabbit and Steel,29.65 SR,NaN,2 GB RAM,128 MB,300 MB available space,2024
250,MuvLuv Alternative,64.12 SR,NaN,2 GB RAM,1024x768,4 GB available space,2017
532,MrMine,No Price,NaN,256 MB RAM,NaN,"512 MB available space * Starting January 1st,...",2020
...,...,...,...,...,...,...,...
8328,TouHou Makuka Sai Fantastic Danmaku Festival,22.88 SR,NaN,1 GB RAM,DirectX 9 compatible graphics card,600 MB,2018
8347,Seen,1.37 SR,NaN,4 GB RAM,1GB of memory or higher,1 GB,2019
8408,Zombie Estate 2,No Price,NaN,4 GB RAM,GPU capable of Shader Model 2 and DirectX 9.0c...,300 MB,2017
8421,9 Childs Street,No Price,NaN,8 GB RAM,gtx 1050 \ rx 560,4 GB,2023


In [306]:
# dropping variable for rows where Processor, Graphics and storage are nulls
col_drop5 = (df_steam['Processor'].isnull()) & (df_steam['Graphics'].isnull()) & (df_steam['Storage'].isnull())

df_steam.drop(df_steam[col_drop5].index, inplace=True)

In [307]:
# dropping variable for rows where Processor, Graphics are nulls
col_drop6 = (df_steam['Processor'].isnull()) & (df_steam['Graphics'].isnull())

df_steam.drop(df_steam[col_drop6].index, inplace=True)

In [308]:
# now we want to fill in missing Processor values based on the most common processor type for a specific memory configuration

# calculate the mode of the 'Processor' for each 'Memory' group
processor_mode_per_memory = df_steam.groupby('Memory')['Processor'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

# fill the missing 'Processor' values with the calculated mode
df_steam['Processor'].fillna(processor_mode_per_memory, inplace=True)

In [309]:
df_steam[df_steam['Processor'].isnull()]

,Game Names,Price,Processor,Memory,Graphics,Storage,years


In [310]:
df_steam[df_steam['Processor'] == 'and operating system']

,Game Names,Price,Processor,Memory,Graphics,Storage,years
3,The Riftbreaker Heart of the Swamp,Your Price:106.00 SR,and operating system,8 GB RAM,Nvidia GTX 750 2GB or AMD R7 265 2GB,12 GB available space Sound Card:,2024
9,Endzone A World Apart Distant Places,10.77 SR,and operating system,8 GB RAM,GeForce GTX 760 or equivalent,6 GB available space,2022
10,Motor Town Behind The Wheel,No Price,and operating system,4 GB RAM,"GeForce GTX 750 4GB, Radeon RX 460 4GB",5 GB available space,2021
11,WARNO Digital Guides,333.27 SR,and operating system,4 GB RAM,Nvidia GeForce GT 1030 (Legacy: Nvidia GeForce...,50 GB available space Sound Card:,2024
17,Broforce,No Price,and operating system,4 GB RAM,OpenGL 3.0 compliant video card,1300 MB available space,2015
...,...,...,...,...,...,...,...
8227,The RPG Engine GameMasters Edition REQUIRES B...,56.96 SR,and operating system,6 GB RAM,DirectX 11.1 GPU,6 GB,2023
8328,TouHou Makuka Sai Fantastic Danmaku Festival,22.88 SR,and operating system,1 GB RAM,DirectX 9 compatible graphics card,600 MB,2018
8347,Seen,1.37 SR,and operating system,4 GB RAM,1GB of memory or higher,1 GB,2019
8408,Zombie Estate 2,No Price,and operating system,4 GB RAM,GPU capable of Shader Model 2 and DirectX 9.0c...,300 MB,2017


In [311]:
df_steam.isnull().sum()

Game Names      0
Price           0
Processor       0
Memory        197
Graphics      683
Storage       610
years           0
dtype: int64

> 2. Memory: 397

In [312]:
df_steam['Memory'].isnull().sum()

197

In [313]:
# checking null values of memory with other features
df_steam[(df_steam['Memory'].isnull()) & (df_steam['Storage'].isnull()) & (df_steam['Graphics'].isnull())]


,Game Names,Price,Processor,Memory,Graphics,Storage,years
278,BioShock,No Price,at 2.4GHz RAM : 1 GB Video Card : Direct X 9.0...,NaN,NaN,NaN,2007
300,The Last of Us Part I Upgrade to Digital Delu...,No Price,and operating system,NaN,NaN,NaN,No Release Date
338,Spirit Island Branch Claw,8.99 SR,and operating system,NaN,NaN,NaN,2021
418,Car Mechanic Simulator 2021 Jeep RAM Remaste...,Your Price:663.45 SR,and operating system,NaN,NaN,NaN,2024
521,Business Tour Board Game with Online Multiplayer,No Price,and operating system,NaN,NaN,NaN,2017
531,HalfLife Opposing Force,Your Price:83.86 SR,", 96mb ram, 16mb video card, Windows XP, Mouse...",NaN,NaN,NaN,1999
719,shapez Puzzle DLC,Your Price:101.04 SR,and operating system Network: Broadband Intern...,NaN,NaN,NaN,2021
757,HalfLife Blue Shift,Your Price:83.86 SR,", 96mb ram, 16mb video card, Windows XP, Mouse...",NaN,NaN,NaN,2001
836,Cataclysm Dark Days Ahead,No Price,and operating system,NaN,NaN,NaN,2023
934,Warhammer 40000 Dawn of War Dark Crusade,No Price,", 512 MB RAM, 3.5 GB free hard drive space, 64 MB",NaN,NaN,NaN,2006


In [314]:
# dropping variable for rows where Memory, Graphics and Storage are nulls
col_drop7 = (df_steam['Memory'].isnull()) & (df_steam['Graphics'].isnull()) & (df_steam['Storage'].isnull())

df_steam.drop(df_steam[col_drop7].index, inplace=True)

In [315]:
df_steam['Memory'].isnull().sum()

161

In [316]:
# variable for rows where Memory and Storage are nulls
df_steam[(df_steam['Memory'].isnull()) & (df_steam['Storage'].isnull())]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
43,HITMAN 3 Deluxe Pack,No Price,Intel CPU Core i5-2500K 3.3GHz / AMD CPU Pheno...,NaN,NVIDIA GeForce GTX 660 / Radeon HD 7870,NaN,No Release Date
162,Lossless Scaling,No Price,and operating system,NaN,Intel HD,NaN,2018
282,HalfLife 2 Episode One,Your Price:83.86 SR,", 512MB RAM,",NaN,"Card (Requires support for SSE), Windows® 7 (3...",NaN,2006
285,Northgard Nidhogg Clan of the Dragon,5.97 SR,Intel 2.0ghz Core 2 Duo or equivalent,NaN,Nvidia 450 GTS / Radeon HD 5750 or better,NaN,2018
485,Hammerwatch Anniversary Edition,37.48 SR,and operating system,NaN,OpenGL 3.2+ /,NaN,2023
495,HalfLife 2 Episode Two,Your Price:83.86 SR,", 512MB RAM,",NaN,"Card (Requires support for SSE), Windows® 7 (3...",NaN,2007
767,Graveyard Keeper Better Save Soul,115.89 SR,"Intel core i5, 1.5 GHz and up",NaN,"1 Gb dedicated video card, shader model 3.0+ *...",NaN,2021
1046,Graveyard Keeper Stranger Sins,115.89 SR,"Intel core i5, 1.5 GHz and up",NaN,"1 Gb dedicated video card, shader model 3.0+ *...",NaN,2019
1159,LEGO Harry Potter Years 14,15.00 SR,Intel P4 1.8 GHz/AMD Athlon XP 2200+ RAM: 512MB,NaN,Card: ATI X1300/NVIDIA GeForce FX 5800 (256 MB...,NaN,2010
1270,Until Dawn,No Price,and operating system,NaN,- Sound Card: -,NaN,No Release Date


In [317]:
# dropping variable for rows where Memory and Storage are nulls
col_drop8 = (df_steam['Memory'].isnull()) & (df_steam['Storage'].isnull())

df_steam.drop(df_steam[col_drop8].index, inplace=True)

In [318]:
df_steam['Memory'].isnull().sum()

124

In [319]:
# variable for rows where Memory and Graphics are nulls
df_steam[(df_steam['Memory'].isnull()) & (df_steam['Graphics'].isnull())]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
104,Feign,No Price,CPU: SSE2 instruction set support.,NaN,NaN,"800 MB available space * Starting January 1st,...",2021
153,Untitled Goose Game,22.49 SR,and operating system,NaN,NaN,830 MB available space,2020
288,Feign,No Price,CPU: SSE2 instruction set support.,NaN,NaN,"800 MB available space * Starting January 1st,...",2021
793,Crocotile 3D,No Price,An Intel Pentium 4,NaN,NaN,"250 MB available space * Starting January 1st,...",2015
1035,The Game of Life 2 The Ultimate Life Collection,76.06 SR,2 GHz single core,NaN,NaN,2 MB available space Sound Card: Integrated so...,2020
1056,Crocotile 3D,No Price,An Intel Pentium 4,NaN,NaN,"250 MB available space * Starting January 1st,...",2015
1200,Dungeons Degenerate Gamblers,Your Price:79.54 SR,and operating system,NaN,NaN,200 MB available space,2024
1414,Need for Speed Most Wanted Complete DLC Bundle,No Price,2 GHz Dual Core (Core 2 Duo 2.4 GHZ or Althon ...,NaN,NaN,2 GB available space Sound Card:,2020
1491,Lingo,No Price,and operating system,NaN,NaN,53 MB available space,2021
1524,BlazBlue Entropy Effect Soundtrack A,Your Price:63.70 SR,and operating system,NaN,NaN,200 MB available space,2024


In [320]:
# dropping variable for rows where Memory and Storage are nulls
col_drop9 = (df_steam['Memory'].isnull()) & (df_steam['Graphics'].isnull())

df_steam.drop(df_steam[col_drop9].index, inplace=True)

In [321]:
df_steam['Memory'].isnull().sum()

90

In [322]:
# checking unique values of other features for null memory
null_memory_rows = df_steam[df_steam['Memory'].isnull()]
unique_processors = null_memory_rows['Processor'].unique()
unique_storage = null_memory_rows['Storage'].unique()

unique_storage
#unique_processors


array(['250 MB available space Sound Card:', '1500 MB available space',
       '12 GB available space Sound Card:', '500 MB available space',
       '35 GB available space', '10 GB available space',
       '324 MB available space * Starting January 1st, 2024, the Steam Client will only support Windows 10 and later versions. View all What Curators Say 27 Curators have reviewed this product. Click here to see them. Customer reviews for Prodigal Overall Reviews: Very P',
       '15 GB available space', '1 GB available space',
       '10 GB available space * Starting January 1st, 2024, the Steam Client will only support Windows 10 and later versions. View all What Curators Say 3 Curators have reviewed this product. Click here to see them. Customer reviews for Zombie Within Overall Reviews: M',
       '10 MB available space', '17 GB available space Sound Card:',
       '6 GB available space', '12 GB available space',
       '300 MB available space',
       '4 GB available space Sound Card: 

In [323]:
# Filter most frequent value of Storage where 'Memory' is null
null_memory_df = df_steam[df_steam['Memory'].isnull()]
most_frequent_storage = null_memory_df['Storage'].mode()[0]

In [324]:
df_steam[(df_steam['Storage'] == '8 GB ') & df_steam['Memory'].isnull()]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
5255,Valhalla Path Survival,21.40 SR,2.4 Ghz Dual Core CPU,NaN,2 GB Dedicated Memory,8 GB,2024
5890,Escape Memoirs Mansion Heist,61.46 SR,Intel i3 / AMD FX series or equivalent,NaN,GTX 760M / GTX 660 / Ati Radeon 7870 or simila...,8 GB,2022
6004,RPG Developer Bakin PixelScapes Town Pack,55.34 SR,Intel Core i3-4340 or better,NaN,Graphic board that works with OpenGL 4.4 or hi...,8 GB,2024
6299,Escape Memoirs Mini Stories,5.96 SR,Intel i3 / AMD FX series or equivalent,NaN,GTX 760M / GTX 660 / Ati Radeon 7870 or simila...,8 GB,2022
7323,RPG Developer Bakin PixelScapes Town Pack,55.34 SR,Intel Core i3-4340 or better,NaN,Graphic board that works with OpenGL 4.4 or hi...,8 GB,2024
7488,,No Price,Intel® Core™ i3,NaN,NVIDIA® GeForce® GTX 760 or AMD Radeon™ R7 260...,8 GB,2024
7549,Escape Memoirs Mini Stories Bunker Scenario,5.96 SR,Intel i3 / AMD FX series or equivalent,NaN,GTX 760M / GTX 660 / Ati Radeon 7870 or simila...,8 GB,2023


In [325]:
### impute of memory is not possible because it's not convienet to have a memory of 8 GB RAM and storage of 8 GB!


In [326]:
# # 1: '8 GB '
# # now we will calculate the mode of 'Memory' where 'Storage' is '8 GB ' and 'Memory' is not null
# mode_memory_8gb = df_steam[(df_steam['Storage'] == '8 GB ') & (~df_steam['Memory'].isnull())]['Memory'].mode()[0]

# # impute the 'Memory' feature where 'Storage' is '8 GB ' and 'Memory' is null
# df_steam.loc[(df_steam['Storage'] == '8 GB ') & (df_steam['Memory'].isnull()), 'Memory'] = mode_memory_8gb

# df_steam[(df_steam['Storage'] == '8 GB ') & (df_steam['Memory'] == mode_memory_8gb)]

In [327]:
# we need to drop the rest of them 
df_steam.dropna(subset=['Memory'], inplace=True)

In [328]:
df_steam.isnull().sum()

Game Names      0
Price           0
Processor       0
Memory          0
Graphics      613
Storage       537
years           0
dtype: int64

In [329]:
df_steam.shape

(8145, 7)

> 3. Graphics: 673

In [330]:
# check nulls in Graphics
df_steam['Graphics'].isnull().sum()

613

In [331]:
# check nulls of graphics and storage
df_steam[(df_steam['Graphics'].isnull()) & (df_steam['Storage'].isnull())]  # 87 rows, need to drop

,Game Names,Price,Processor,Memory,Graphics,Storage,years
38,LEGO Star Wars III The Clone Wars,Your Price:491.48 SR,Intel P4 2.24 GHz or AMD Athlon 64,512 MB RAM Hard Disk Space: 8GB Video Card: 25...,NaN,NaN,2011
127,Gone Home,9.88 SR,• 2GB,• DX10 Video card with 512MB of VRAM • (NOTE:...,NaN,NaN,2013
199,The Lord of the Rings Online,No Price,Core 2 Duo / AMD Athlon II or better,2GB Hard Disk Space: 32GB for high resolution*...,NaN,NaN,2012
200,The Lord of the Rings Online,No Price,Core 2 Duo / AMD Athlon II or better,2GB Hard Disk Space: 32GB for high resolution*...,NaN,NaN,2012
276,Gone Home,9.88 SR,• 2GB,• DX10 Video card with 512MB of VRAM • (NOTE:...,NaN,NaN,2013
...,...,...,...,...,...,...,...
4336,War Thunder Chinese Starter Pack,No Price,Dual-Core 2.2 GHz,4GB (Windows Vista/7/8/10),NaN,NaN,2022
4359,Duke Nukem Forever,No Price,Intel Core 2 Duo @ 2.0 Ghz / AMD Athlon 64 X2 ...,1 Gb Hard Disk Space: 10 Gb free Video,NaN,NaN,No Release Date
4362,Kings Quest Collection,No Price,1.0 GHz,256 MB RAM,NaN,NaN,2006
4449,SuperPower 2 Steam Edition,No Price,1.5 GHz CPU,256 MB RAM,NaN,NaN,2004


In [332]:
# dropping variable for rows where Memory and Storage are nulls
col_drop10 = (df_steam['Graphics'].isnull()) & (df_steam['Storage'].isnull())

df_steam.drop(df_steam[col_drop10].index, inplace=True)

In [333]:
df_steam['Graphics'].isnull().sum()

526

In [334]:
# Filter most frequent value of Storage where 'Graphics' is null
null_graphics_df = df_steam[df_steam['Graphics'].isnull()]
most_frequent_storage2 = null_graphics_df['Storage'].mode()[0]
most_frequent_storage2

'1 GB available space'

In [335]:
#df_steam[(df_steam['Graphics'].isnull()) & (df_steam['Storage'] == '1 GB available space')]

In [336]:
# # 1: check '1 GB available space'
# # there are some rows with storage '1 GB available space' and have nulls in Graphics
# df_steam[(df_steam['Graphics'].isnull()) & (df_steam['Storage'] == '1 GB available space')].value_counts().sum()

# # now we will calculate the mode of 'Graphics' where 'Storage' is '1 GB available space' is null
# mode_graphics_1gb = df_steam[(df_steam['Storage'] == '1 GB available space') & (~df_steam['Graphics'].isnull())]['Graphics'].mode()[0]

# # impute the 'Graphics' feature where 'Storage' is '1 GB available space' and 'Graphics' is null
# df_steam.loc[(df_steam['Storage'] == '1 GB available space') & (df_steam['Graphics'].isnull()), 'Graphics'] = mode_graphics_1gb

# df_steam[(df_steam['Storage'] == '1 GB available space') & (df_steam['Graphics'] == mode_graphics_1gb)]

In [337]:
# The Intel Core 2 Duo is a very old processor, 
# first released in 2006. By 2022, this processor would be considered highly outdated and unlikely to run most modern games, even at minimum settings.
# Modern games typically require at least a mid-range processor from the last 5-10 years.



In [338]:
# Graphics cards are mostly have relationship more with processor type, so it's logical to fill it using its mode
# impute 'Graphics' based on the mode within 'Processor' groups
df_steam['Graphics'] = df_steam.groupby('Processor')['Graphics'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))
df_steam['Graphics'].isnull().sum()

153

In [339]:
# checking the unique values of processor related to the nulls in Graphics
graph_null = df_steam[df_steam['Graphics'].isnull()]
processor_value_counts = graph_null['Processor'].value_counts()
total_processor_count = processor_value_counts.sum()
# there are 138 unique processors for the Graphics nulls, this would make noise on the data so
# i think the best thing is to drop them
total_processor_count

153

In [340]:
# dropping the nulls 138 in graphics
df_steam = df_steam.drop(graph_null.index)


In [341]:
# checking
df_steam['Graphics'].isnull().sum()

0

In [342]:
df_steam.isnull().sum()

Game Names      0
Price           0
Processor       0
Memory          0
Graphics        0
Storage       450
years           0
dtype: int64

> 4. Storage:   450

In [343]:
df_steam['Storage'].isnull().sum()

450

In [344]:
# filter rows where 'Storage' is null
storage_null = df_steam[df_steam['Storage'].isnull()]

# find unique values in other columns where 'Storage' is null
unique_memory_values = storage_null['Memory'].unique()
unique_processor_values = storage_null['Processor'].unique()
unique_graphics_values = storage_null['Graphics'].unique()
#unique_memory_values
#unique_processor_values
unique_graphics_values

array(['Nvidia GeForce 6XXX series or higher; AMD Radeon 1XXX series or higher',
       'Card with Shader Model 3 support, 256 MB video',
       'Nvidia® GeForce™ GTX 460 / AMD® Radeon™ HD 5850 Video',
       'DX10 (SM 4.0) capable, 512MB VRAM Hard Drive: 3 GB HD space',
       'Nvidia Geforce 7800 GTX with 256MB of VRAM or ATI Radeon HD 2600 Pro with 256MB of VRAM Sound: Direct X 9.0c compatible sound card',
       '256 MB VRAM or better and must support 1024x768 or higher resolution in OpenGL ***IMPORTANT*** Not all Intel integrated video cards will play the game.',
       'NVIDIA GTX 650 2GB or AMD HD7770 2GB',
       'Nvidia GeForce 8600GT (256MB), AMD/ATI Radeon HD3650 (256MB), Intel HD3000 or above',
       'NVIDIA® GeForce® GTX 960 / GTX 1650 or AMD Radeon™ RX 470',
       '256 MB Video Card w/Pixel Shader 3.0 Support (Radeon X1650 / GeForce 6800* or Better)',
       '2 GB VRAM, OpenGL 3.3',
       "ATI Radeon HD 2400 or NVIDIA GeForce 7600 or better Hard Drive: 1.5 GB OpenGL: 2

In [345]:
# Imputation 1: if 'Memory' is low (ex: '1 GB RAM'), assume small 'Storage'
df_steam.loc[(df_steam['Memory'] == '1 GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '2 GB available space'
df_steam.loc[(df_steam['Memory'] == '4 GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '20 GB available space'
df_steam.loc[(df_steam['Memory'] == '2GB or more Hard Drive: At least 7GB of free space') & (df_steam['Storage'].isnull()), 'Storage'] = '7 GB available space'
df_steam.loc[(df_steam['Memory'] == '2 GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '8 GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '30 GB available space'
df_steam.loc[(df_steam['Memory'] == '2 GB Available System') & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '1 GB') & (df_steam['Storage'].isnull()), 'Storage'] = '2 GB available space'
df_steam.loc[(df_steam['Memory'] == '16 GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '50 GB available space'
df_steam.loc[(df_steam['Memory'] == '1 GB RAM (2 GB RAM recommended) Sound:') & (df_steam['Storage'].isnull()), 'Storage'] = '5 GB available space'
df_steam.loc[(df_steam['Memory'] == '512 MB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '3 GB available space'
df_steam.loc[(df_steam['Memory'] == '3GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '15 GB available space'
df_steam.loc[(df_steam['Memory'] == '1 GB RAM (XP), 1.5 GB RAM (Vista)') & (df_steam['Storage'].isnull()), 'Storage'] = '6 GB available space'
df_steam.loc[(df_steam['Memory'] == '2 GB for 32-bit') & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '2GB System RAM or more') & (df_steam['Storage'].isnull()), 'Storage'] = '12 GB available space'
df_steam.loc[(df_steam['Memory'] == '6 GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '25 GB available space'
df_steam.loc[(df_steam['Memory'] == '2GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '1GB GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '2 GB available space'
df_steam.loc[(df_steam['Memory'] == '2GB') & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '3 GB RAM') & (df_steam['Storage'].isnull()), 'Storage'] = '15 GB available space'
df_steam.loc[(df_steam['Memory'] == '1 GB (2 GB recommended)') & (df_steam['Storage'].isnull()), 'Storage'] = '2 GB available space'
df_steam.loc[(df_steam['Memory'] == '2 GB') & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'

# Imputation 2: if 'Graphics' indicates a high-end game, assume larger 'Storage'
df_steam.loc[(df_steam['Graphics'].str.contains('NVIDIA GTX', case=False)) & (df_steam['Storage'].isnull()), 'Storage'] = '50 GB available space'
df_steam.loc[(df_steam['Graphics'] == 'NVIDIA GeForce 9800 GT / ATI Radeon HD 5750, 512 MB video RAM Hard Drive: 11 GB Sound: Windows compatible sound card') & (df_steam['Storage'].isnull()), 'Storage'] = '15 GB available space'

# Imputation 3: based on years (here i searched about the game and found a relation with the memory related to the updates of version, and made gpt organize it)
df_steam.loc[(df_steam['Memory'].str.contains('256 MB RAM, 512 MB RAM required for Windows Vista', regex=False)) & (df_steam['years'] == 2008) & (df_steam['Storage'].isnull()), 'Storage'] = '5 GB available space'
df_steam.loc[(df_steam['Memory'].str.contains('1GB Ram (Windows XP), 2GB Ram (Windows Vista, Windows 7)', regex=False)) & (df_steam['years'] == 2013) & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '4GB RAM') & (df_steam['years'] == 2018) & (df_steam['Storage'].isnull()), 'Storage'] = '25 GB available space'
df_steam.loc[(df_steam['Memory'] == '1+ gigs of RAM GB RAM') & (df_steam['Graphics'].str.contains('256 MB or higher', regex=False)) & (df_steam['years'] == 2014) & (df_steam['Storage'].isnull()), 'Storage'] = '15 GB available space'
df_steam.loc[(df_steam['Memory'] == '1GB') & (df_steam['Graphics'].str.contains('nVidia GeForce 7800/7900/8600 series', regex=False)) & (df_steam['years'] == 2011) & (df_steam['Storage'].isnull()), 'Storage'] = '20 GB available space'
df_steam.loc[(df_steam['Memory'] == '1.5 GB RAM') & (df_steam['years'] == 'No Release Date') & (df_steam['Storage'].isnull()), 'Storage'] = '8 GB available space'
df_steam.loc[(df_steam['Memory'] == '1.5 GB Windows XP / 2 GB Windows Vista, Windows 7') & (df_steam['years'] == 2010) & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '2GB RAM (XP)/3GB RAM (Windows 7 / Vista)') & (df_steam['years'] == 2012) & (df_steam['Storage'].isnull()), 'Storage'] = '20 GB available space'
df_steam.loc[(df_steam['Memory'] == '1 GB of RAM') & (df_steam['years'] == 2008) & (df_steam['Storage'].isnull()), 'Storage'] = '5 GB available space'
df_steam.loc[(df_steam['Memory'] == '256 MB RAM') & (df_steam['years'] == 2006) & (df_steam['Storage'].isnull()), 'Storage'] = '4 GB available space'
df_steam.loc[(df_steam['Memory'].str.contains('1GB RAM Hard Disk Space: 1.2 GB available space', regex=False)) & (df_steam['years'] == 2012) & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '1 GB (1.5 GB Vista and Windows 7) Hard Drive:') & (df_steam['years'] == 2010) & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '1536 MB RAM') & (df_steam['years'] == 2009) & (df_steam['Storage'].isnull()), 'Storage'] = '12 GB available space'
df_steam.loc[(df_steam['Memory'] == '96 MB RAM (128 MB RAM or greater recommended)') & (df_steam['years'] == 'No Release Date') & (df_steam['Storage'].isnull()), 'Storage'] = '2 GB available space'
df_steam.loc[(df_steam['Memory'] == '1GB RAM Hard Disk Space: 1.2 GB available space') & (df_steam['years'] == 'No Release Date') & (df_steam['Storage'].isnull()), 'Storage'] = '8 GB available space'


In [346]:
df_steam['Storage'].isnull().sum()

58

In [347]:
null_storage_rows = df_steam[df_steam['Storage'].isnull()]

# Display the rows
null_storage_rows[1:58]

,Game Names,Price,Processor,Memory,Graphics,Storage,years
362,Call of Duty 2,"Your Price:3,498.30 SR","or higher, 256MB RAM (512MB RAM recommended), ...",512 MB RAM Hard Disk Space: 4 GB,NVIDIA Geforce 7300 or better or ATI Radeon X1...,NaN,2005
376,MechWarrior 5 Mercenaries Call to Arms,Your Price:110.34 SR,and operating system,8 MB RAM,NVIDIA GeForce GTX 770/AMD Radeon R9 280X,NaN,2022
479,LEGO Batman The Videogame,No Price,Intel® P3 1.5 GHz or AMD Athlon™ XP,"256 MB RAM, 512 MB RAM required for Windows Vi...",128 MB with Shader Model 2.0 capability (Shade...,NaN,2008
609,Sonic AllStars Racing Transformed Collection,Your Price:801.81 SR,2GHz Intel Core 2 Duo or equivalent,"1GB Ram (Windows XP), 2GB Ram (Windows Vista, ...",256MB with Shader Model 3.0 support,NaN,2013
633,Total War ROME II Rise of the Republic Campai...,319.50 SR,2nd Generation Intel Core i5,4GB RAM,1024 MB,NaN,2018
806,Dont Starve Reign of Giants,Your Price:76.76 SR,1.7+ GHz or better,1+ gigs of RAM GB RAM,Radeon HD5450 or better; 256 MB or higher,NaN,2014
926,Serious Sam 3 BFE,Your Price:85.17 SR,Dual-core from Intel or AMD at 2.0 GHz,1GB,"nVidia GeForce 7800/7900/8600 series, ATI/AMD ...",NaN,2011
1478,Tom Clancys Splinter Cell Conviction,No Price,1.8 GHz Intel Core2 Duo or 2.4 GHz AMD Athlon ...,"1.5 GB Windows XP / 2 GB Windows Vista, Windows 7",256 MB,NaN,2010
1530,Binary Domain,No Price,Intel Core 2 Duo @ 2.66 GHz or AMD equivalent,2GB RAM (XP)/3GB RAM (Windows 7 / Vista),NVIDIA GeForce GT220 (512MB) / ATI Radeon HD 2...,NaN,2012
1645,Tom Clancys Rainbow Six Vegas 2,Your Price:327.19 SR,3 GHz Pentium 4 or AMD Athlon 3000 (3.5 GHz Pe...,1 GB of RAM,128 MB,NaN,2008


In [348]:
df_steam['Storage'].isnull().sum()

58

In [349]:
df_steam.isnull().sum()

Game Names     0
Price          0
Processor      0
Memory         0
Graphics       0
Storage       58
years          0
dtype: int64

In [350]:
# it's not imputed idk why
df_steam.loc[(df_steam['Memory'].str.contains('1 GB RAM Hard Disk Space', regex=False)) & (df_steam['years'] == 2011) & (df_steam['Storage'].isnull()), 'Storage'] = '2 GB available space'
df_steam.loc[(df_steam['Memory'].str.contains('512 MB RAM Hard Disk Space', regex=False)) & (df_steam['years'] == 2005) & (df_steam['Storage'].isnull()), 'Storage'] = '4 GB available space'
df_steam.loc[(df_steam['Memory'].str.contains('8 MB RAM', regex=False)) & (df_steam['years'] == 2022) & (df_steam['Storage'].isnull()), 'Storage'] = '1 GB available space'
df_steam.loc[(df_steam['Memory'].str.contains('256 MB RAM', regex=False)) & (df_steam['years'] == 2008) & (df_steam['Storage'].isnull()), 'Storage'] = '5 GB available space'
df_steam.loc[(df_steam['Memory'].str.contains('1GB Ram', regex=False)) & (df_steam['years'] == 2013) & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'] == '4GB RAM') & (df_steam['years'] == 2018) & (df_steam['Storage'].isnull()), 'Storage'] = '25 GB available space'
df_steam.loc[(df_steam['Memory'].str.contains('1+ gigs of RAM GB RAM', regex=False)) & (df_steam['Graphics'].str.contains('256 MB or higher', regex=False)) & (df_steam['years'] == 2014) & (df_steam['Storage'].isnull()), 'Storage'] = '15 GB available space'
df_steam.loc[(df_steam['Memory'] == '1GB') & (df_steam['Graphics'].str.contains('nVidia GeForce 7800/7900/8600 series', regex=False)) & (df_steam['years'] == 2011) & (df_steam['Storage'].isnull()), 'Storage'] = '20 GB available space'
df_steam.loc[(df_steam['Memory'].str.contains('1.5 GB Windows XP', regex=False)) & (df_steam['years'] == 2010) & (df_steam['Storage'].isnull()), 'Storage'] = '10 GB available space'
df_steam.loc[(df_steam['Memory'].str.contains('2GB RAM (XP)', regex=False)) & (df_steam['years'] == 2012) & (df_steam['Storage'].isnull()), 'Storage'] = '20 GB available space'
df_steam.loc[(df_steam['Memory'] == '1 GB of RAM') & (df_steam['years'] == 2008) & (df_steam['Storage'].isnull()), 'Storage'] = '5 GB available space'
df_steam.loc[(df_steam['Memory'] == '256 MB RAM') & (df_steam['years'] == 2006) & (df_steam['Storage'].isnull()), 'Storage'] = '4 GB available space'


In [351]:
#df_steam[df_steam['Storage'].isnull()]

In [352]:
df_steam.isnull().sum()

Game Names     0
Price          0
Processor      0
Memory         0
Graphics       0
Storage       58
years          0
dtype: int64

In [353]:
# i'm gonna drop it and add it to another dataframe to complete dealing with it

df_steam_test = df_steam.dropna(subset=['Storage'])

In [354]:
df_steam_test.isnull().sum()

Game Names    0
Price         0
Processor     0
Memory        0
Graphics      0
Storage       0
years         0
dtype: int64

In [355]:
df_steam.isnull().sum()

Game Names     0
Price          0
Processor      0
Memory         0
Graphics       0
Storage       58
years          0
dtype: int64

In [356]:
df_steam_test

,Game Names,Price,Processor,Memory,Graphics,Storage,years
0,Kingdom Rush Origins Tower Defense,Your Price:248.95 SR,Dual Core CPU,1 GB RAM,OpenGL 3.0 compliant with 512MB of video RAM.,500 MB available space,2018
1,War Thunder Su39 Pack,No Price,"2,2 GHz",1 GB RAM,Nvidia GeForce 6XXX series or higher; AMD Rade...,2 GB available space,2023
2,The Battle of Polytopia,79.04 SR,Dual Core 2.0 Ghz,1 GB RAM,128mb Video,160 MB available space,2020
3,The Riftbreaker Heart of the Swamp,Your Price:106.00 SR,and operating system,8 GB RAM,Nvidia GTX 750 2GB or AMD R7 265 2GB,12 GB available space Sound Card:,2024
4,Soulash,Your Price:62.00 SR,2 cores,500 MB RAM,OpenGL,400 MB available space,2022
...,...,...,...,...,...,...,...
8519,Siege Survival Gloria Victis,118.80 SR,Intel Core i3 or AMD Athlon II X4 with at leas...,4 GB RAM,"GeForce GT 740, Radeon HD 7750 - at least 1GB ...",10 GB,2021
8520,Lay a Beauty to Rest A Butterfly Dream,34.10 SR,Intel Core i3 2.00 GHz 或 AMD A10,2 GB RAM,Intel® HD Graphics 3000,3 GB,2022
8521,Pamali Indonesian Folklore Horror,34.64 SR,Intel(R) Core(TM) i5-7200U CPU @ 2.50Ghz,4 GB RAM,NVIDIA GeForce 940MX,2 GB,2018
8522,A fascinating story,56.04 SR,Intel Core i5,4 GB RAM,NVIDIA Gef750,8 GB,2024


# Dealing with values of features:

> Storage: make it ex: (3 MG) without the other details and remove any spaces \n

> Memory: make it ex: (2 MG RAM) without the other details and remove any spaces

> Graphics: 


> Processor: 

In [357]:
# Storage split:

df_steam_test.loc[:, 'Storage'] = df_steam_test['Storage'].apply(
    lambda x: re.search(r'\d+\s*[MGK]B', str(x), re.IGNORECASE).group(0).strip() if re.search(r'\d+\s*[MGK]B', str(x), re.IGNORECASE) else x
)
# Remove any spaces from the 'Storage' values
df_steam_test.loc[:, 'Storage'] = df_steam_test['Storage'].str.strip()

df_steam_test['Storage'].unique()

array(['500 MB', '2 GB', '160 MB', '12 GB', '400 MB', '15 GB', '150 MB',
       '35 GB', '6 GB', '5 GB', '50 GB', '60 GB', '5000 MB', '20 GB',
       '1300 MB', '75 GB', '1 GB', '64 GB', '25 GB', '7 GB', '24 GB',
       '30 GB', '45 GB', '13 GB', '16 GB', '200 MB', '4 GB', '10 GB',
       '700 MB', '1536 MB', '100 MB', '14 GB', '50 MB', '59 GB', '550 MB',
       '20 MB', '80 GB', '1600 MB', '40 GB', '3 GB', '10 MB', '100 GB',
       '18 GB', '300 MB', '200 GB', '103 GB', '28 GB', '600 MB', '49 GB',
       '250 MB', '150 GB', '36 GB', '9 GB', '1250 MB', '140 GB', '70 GB',
       '83 GB', '41 GB', '1500 MB', '1000 MB', '8 GB', '67 GB', '650 MB',
       '950 MB', '350 MB', '320 MB', '1024 MB', '33 GB', '900 MB',
       '72 GB', '77 GB', '11 GB', '26 GB', '60 MB', '654 MB', '768 MB',
       '90 GB', '23 GB', '22 GB', '738 MB', '12 MB', '2500 MB', '6500 MB',
       '27 GB', '125 GB', '800 MB', '65 GB', '17 GB', '85 GB', '10000 MB',
       '55 GB', '512 MB', '1200 MB', '65 MB', '4200 MB', '1

In [358]:
df_steam_test['Memory'].unique()

array(['1 GB RAM', '8 GB RAM', '500 MB RAM', '3 GB RAM', '4 GB RAM',
       '2 GB RAM', '5 GB RAM', '8192 MB RAM', '10 GB RAM',
       '2GB or more Hard Drive: At least 7GB of free space', '512 MB RAM',
       '6 GB RAM', '2048 MB RAM', '16 GB RAM', '2 GB Available System',
       '1 GB', '200 MB RAM', '1024 MB RAM', '1000 MB RAM', '12 GB RAM',
       '2.5 GB RAM', '2GB RAM', '2 GB for 32-bit', '1 MB RAM',
       '512 MB RAM Network: Broadband Internet connection',
       '2GB System RAM or more',
       '1 GB RAM View all What Curators Say 102 Curators have reviewed this product. Click here to see them. Customer reviews for Rusty Lake Paradise Overall Reviews: Overwhelmingly P',
       '16 MB RAM', '2 GB', '3GB RAM',
       '1 GB RAM (2 GB RAM recommended) Sound:', '8 MB RAM',
       '4000 MB RAM', '2GB', '514 MB RAM',
       '1 GB RAM (XP), 1.5 GB RAM (Vista)', '256 MB RAM', '128 MB RAM',
       '4 MB RAM', '4 GB RAM GB RAM', '6 MB RAM', '8 GB RAM GB RAM',
       '1500 MB RAM', '1.5 

In [359]:
# Memory split:

df_steam_test.loc[:, 'Memory'] = df_steam_test['Memory'].apply(
    lambda x: re.search(r'\d+\s*[MG]B\s*RAM', str(x), re.IGNORECASE).group(0).strip() if re.search(r'\d+\s*[MG]B\s*RAM', str(x), re.IGNORECASE) else x
)

# Remove any spaces from the 'Memory' values
df_steam_test.loc[:, 'Memory'] = df_steam_test['Memory'].str.strip()

df_steam_test['Memory'].unique()

array(['1 GB RAM', '8 GB RAM', '500 MB RAM', '3 GB RAM', '4 GB RAM',
       '2 GB RAM', '5 GB RAM', '8192 MB RAM', '10 GB RAM',
       '2GB or more Hard Drive: At least 7GB of free space', '512 MB RAM',
       '6 GB RAM', '2048 MB RAM', '16 GB RAM', '2 GB Available System',
       '1 GB', '200 MB RAM', '1024 MB RAM', '1000 MB RAM', '12 GB RAM',
       '2GB RAM', '2 GB for 32-bit', '1 MB RAM', '2GB System RAM or more',
       '16 MB RAM', '2 GB', '3GB RAM', '8 MB RAM', '4000 MB RAM', '2GB',
       '514 MB RAM', '256 MB RAM', '128 MB RAM', '4 MB RAM', '6 MB RAM',
       '1500 MB RAM', '64 MB RAM', '1GB GB RAM', '2048 GB RAM',
       '4096 MB RAM', '16GB GB RAM', '1 GB (2 GB recommended)',
       '400 MB RAM', '2 MB RAM', '25 MB RAM', '6144 MB RAM',
       '16(Dual-channel) GB RAM', '3000 MB RAM', '2 RAM GB RAM',
       '４ GB RAM', '96 MB RAM', '8GB', ', capable of OpenGL 2.0+ support',
       'is required to h', '2 GB GB RAM', '1GB XP / 2 Vista GB RAM',
       '8000 MB RAM', '14 GB RAM',

In [360]:
# Graphics split:
unique_graphics = df_steam['Graphics'].unique()
# so many unique values
for value in unique_graphics:
    print(value)
# needa splitting oif we need the feature or not including it in the clustering

OpenGL 3.0 compliant with 512MB of video RAM.
Nvidia GeForce 6XXX series or higher; AMD Radeon 1XXX series or higher
128mb Video
Nvidia GTX 750 2GB or AMD R7 265 2GB
OpenGL
GeForce GTX 1650 4GB VRAM (or equivalent) / AMD RX 560 4GB VRAM (or equivalent)
1280x720 minimum resolution, OpenGL 2.0 Support, recommended dedicated
(
Card with Shader Model 3 support, 256 MB video
GeForce GTX 760 or equivalent
GeForce GTX 750 4GB, Radeon RX 460 4GB
Nvidia GeForce GT 1030 (Legacy: Nvidia GeForce GTS 450), AMD Radeon RX 460 (Legacy: ATI Radeon HD 5570)
NVIDIA® GeForce® 8800GTS or better, ATI Radeon™ HD 4850 or better
NVIDIA GTX 460 1GB | AMD Radeon HD 5770 1GB | Intel HD4000 @720p
GeForce GTX 970 / AMD Radeon RX 480
GPU with 2GB VRAM available.
Nvidia® GeForce™ GTX 460 / AMD® Radeon™ HD 5850 Video
OpenGL 3.0 compliant video card
NVIDIA GTX 1650 Super or AMD Radeon RX 5500 XT, with a minimum of 4GB of VRAM
DX10 (SM 4.0) capable, 512MB VRAM Hard Drive: 3 GB HD space
Open GL 3.1+ Compliant
NVIDIA GeFo

In [361]:

df_steam_test.columns

Index(['Game Names', 'Price', 'Processor', 'Memory', 'Graphics', 'Storage',
       'years'],
      dtype='object')

In [362]:
# Processor split:

unique_processors = df_steam['Processor'].unique()
# so many unique values
for value in unique_processors:
    print(value)

Dual Core CPU
2,2 GHz
Dual Core 2.0 Ghz
and operating system
2 cores
Intel Core or AMD Ryzen
2 GHz
Intel® Core™ 2 Duo 3.0Ghz
1 Ghz Dual-Core CPU
Intel® Core™ 2 Duo 2.4 Ghz or better, AMD Athlon™ X2 2.8 Ghz or better
Intel Core i3-3240 / Ryzen 3 1200
Intel/AMD Quad Core
Intel® Core™ i3-2105 / AMD® FX 4300
Core 2 Duo
1.5Ghz Dual-Core Intel Core i5
Core 2 Dual Core 2.4GHz or AMD equivalent
1.8 GHz
2.4 GHz Dual Core
Intel® Core™ i3 @ 3.2 GHZ
Pentium IV 1.8 Ghz or equivalent speed AMD
2.0 GHz Dual Core
Intel Pentium IV @ 3.0 GHz or AMD Athlon64 3000 + @ 1.8 GHz
Intel Core i3-2100 or AMD equivalent
Intel® Core™  i3-530 or AMD® Phenom™ II X3 720
Dual core from Intel or AMD at 2.8 GHz
2.0 GHz or faster
1.5Ghz or better (x86-64)
Intel Core i7 860, Intel Core i5 750, or AMD FX-4100 (SSE 4.2 support required) Video:
AMD A8-7600 / Intel® Core™ i3-2105
Dual Core from Intel or AMD at 2.8 GHz
Intel 2.4 GHz Core 2 Duo or AMD 2.8 GHz Athlon X2
Dual Core 2.4Ghz
Intel Core 2 Duo, AMD 64x2 or above
2GHz D

In [363]:
# splitting the values of processor based on:
# / 
# | 
# and operating systems


# Ensure you're working on a copy of the DataFrame
df_steam_test = df_steam_test.copy()

# Splitting the values of 'Processor' based on 'and operating system'
df_steam_test[['Processor_main', 'Processor_rest']] = df_steam_test['Processor'].str.split('and operating system', n=1, expand=True)

# Converting 'Processor_rest' to string
df_steam_test['Processor_rest'] = df_steam_test['Processor_rest'].astype(str)

# Splitting 'Processor_rest' based on '|'
processor_split = df_steam_test['Processor_rest'].str.split(r'\|', n=1, expand=True)

# Assigning the first part of the split
df_steam_test['Processor_pipe'] = processor_split[0]

# Checking if the second part exists and filling with NaN if it doesn't
df_steam_test['Processor_rest'] = processor_split[1] if 1 in processor_split.columns else np.nan

# Converting 'Processor_rest' to string again
df_steam_test['Processor_rest'] = df_steam_test['Processor_rest'].astype(str)

# Splitting 'Processor_rest' based on '/'
processor_slash_split = df_steam_test['Processor_rest'].str.split(r'/', n=1, expand=True)

# Assigning the split values to the DataFrame, with handling for missing parts
df_steam_test['Processor_slash_1'] = processor_slash_split[0]
df_steam_test['Processor_slash_2'] = processor_slash_split[1] if 1 in processor_slash_split.columns else np.nan

# Selecting the relevant columns
df_steam_test = df_steam_test[['Game Names', 'Price', 'Processor', 'Processor_main', 'Processor_pipe', 'Processor_slash_1', 'Processor_slash_2', 'Memory', 'Graphics', 'Storage', 'years']]

# Displaying the DataFrame
df_steam_test






,Game Names,Price,Processor,Processor_main,Processor_pipe,Processor_slash_1,Processor_slash_2,Memory,Graphics,Storage,years
0,Kingdom Rush Origins Tower Defense,Your Price:248.95 SR,Dual Core CPU,Dual Core CPU,None,nan,NaN,1 GB RAM,OpenGL 3.0 compliant with 512MB of video RAM.,500 MB,2018
1,War Thunder Su39 Pack,No Price,"2,2 GHz","2,2 GHz",None,nan,NaN,1 GB RAM,Nvidia GeForce 6XXX series or higher; AMD Rade...,2 GB,2023
2,The Battle of Polytopia,79.04 SR,Dual Core 2.0 Ghz,Dual Core 2.0 Ghz,None,nan,NaN,1 GB RAM,128mb Video,160 MB,2020
3,The Riftbreaker Heart of the Swamp,Your Price:106.00 SR,and operating system,,,nan,NaN,8 GB RAM,Nvidia GTX 750 2GB or AMD R7 265 2GB,12 GB,2024
4,Soulash,Your Price:62.00 SR,2 cores,2 cores,None,nan,NaN,500 MB RAM,OpenGL,400 MB,2022
...,...,...,...,...,...,...,...,...,...,...,...
8519,Siege Survival Gloria Victis,118.80 SR,Intel Core i3 or AMD Athlon II X4 with at leas...,Intel Core i3 or AMD Athlon II X4 with at leas...,None,nan,NaN,4 GB RAM,"GeForce GT 740, Radeon HD 7750 - at least 1GB ...",10 GB,2021
8520,Lay a Beauty to Rest A Butterfly Dream,34.10 SR,Intel Core i3 2.00 GHz 或 AMD A10,Intel Core i3 2.00 GHz 或 AMD A10,None,nan,NaN,2 GB RAM,Intel® HD Graphics 3000,3 GB,2022
8521,Pamali Indonesian Folklore Horror,34.64 SR,Intel(R) Core(TM) i5-7200U CPU @ 2.50Ghz,Intel(R) Core(TM) i5-7200U CPU @ 2.50Ghz,None,nan,NaN,4 GB RAM,NVIDIA GeForce 940MX,2 GB,2018
8522,A fascinating story,56.04 SR,Intel Core i5,Intel Core i5,None,nan,NaN,4 GB RAM,NVIDIA Gef750,8 GB,2024


In [364]:
df_steam_test['Processor_main'].isnull().sum()

0

In [365]:
df_steam_test['Processor_pipe'].isnull().sum()

0

In [366]:
df_steam_test['Processor_slash_1'].isnull().sum()

0

In [367]:
df_steam_test['Processor_slash_2'].isnull().sum()

7847

In [368]:
# they are all null so we drop the columns
df_steam_test = df_steam_test.drop(['Processor_pipe', 'Processor_slash_1', 'Processor_slash_2'], axis=1)


In [369]:
df_steam_test.head()

,Game Names,Price,Processor,Processor_main,Memory,Graphics,Storage,years
0,Kingdom Rush Origins Tower Defense,Your Price:248.95 SR,Dual Core CPU,Dual Core CPU,1 GB RAM,OpenGL 3.0 compliant with 512MB of video RAM.,500 MB,2018
1,War Thunder Su39 Pack,No Price,"2,2 GHz","2,2 GHz",1 GB RAM,Nvidia GeForce 6XXX series or higher; AMD Rade...,2 GB,2023
2,The Battle of Polytopia,79.04 SR,Dual Core 2.0 Ghz,Dual Core 2.0 Ghz,1 GB RAM,128mb Video,160 MB,2020
3,The Riftbreaker Heart of the Swamp,Your Price:106.00 SR,and operating system,,8 GB RAM,Nvidia GTX 750 2GB or AMD R7 265 2GB,12 GB,2024
4,Soulash,Your Price:62.00 SR,2 cores,2 cores,500 MB RAM,OpenGL,400 MB,2022


In [370]:
# there are empty cells of the and operating system
# needs to fill them again 
df_steam_test[df_steam_test['Processor_main'] =='']

,Game Names,Price,Processor,Processor_main,Memory,Graphics,Storage,years
3,The Riftbreaker Heart of the Swamp,Your Price:106.00 SR,and operating system,,8 GB RAM,Nvidia GTX 750 2GB or AMD R7 265 2GB,12 GB,2024
9,Endzone A World Apart Distant Places,10.77 SR,and operating system,,8 GB RAM,GeForce GTX 760 or equivalent,6 GB,2022
10,Motor Town Behind The Wheel,No Price,and operating system,,4 GB RAM,"GeForce GTX 750 4GB, Radeon RX 460 4GB",5 GB,2021
11,WARNO Digital Guides,333.27 SR,and operating system,,4 GB RAM,Nvidia GeForce GT 1030 (Legacy: Nvidia GeForce...,50 GB,2024
17,Broforce,No Price,and operating system,,4 GB RAM,OpenGL 3.0 compliant video card,1300 MB,2015
...,...,...,...,...,...,...,...,...
8227,The RPG Engine GameMasters Edition REQUIRES B...,56.96 SR,and operating system,,6 GB RAM,DirectX 11.1 GPU,6 GB,2023
8328,TouHou Makuka Sai Fantastic Danmaku Festival,22.88 SR,and operating system,,1 GB RAM,DirectX 9 compatible graphics card,600 MB,2018
8347,Seen,1.37 SR,and operating system,,4 GB RAM,1GB of memory or higher,1 GB,2019
8408,Zombie Estate 2,No Price,and operating system,,4 GB RAM,GPU capable of Shader Model 2 and DirectX 9.0c...,300 MB,2017


In [371]:
# make it null to fill it later in an easy way
df_steam_test['Processor_main'].replace('', np.nan, inplace=True)


In [373]:
# calculate the mode of the 'Processor' for each 'Memory' group
processor_mode_per_graphics = df_steam_test.groupby('Graphics')['Processor_main'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

# fill the missing 'Processor' values with the calculated mode
df_steam_test['Processor_main'].fillna(processor_mode_per_graphics, inplace=True)


In [376]:
df_steam_test.head()

,Game Names,Price,Processor,Processor_main,Memory,Graphics,Storage,years
0,Kingdom Rush Origins Tower Defense,Your Price:248.95 SR,Dual Core CPU,Dual Core CPU,1 GB RAM,OpenGL 3.0 compliant with 512MB of video RAM.,500 MB,2018
1,War Thunder Su39 Pack,No Price,"2,2 GHz","2,2 GHz",1 GB RAM,Nvidia GeForce 6XXX series or higher; AMD Rade...,2 GB,2023
2,The Battle of Polytopia,79.04 SR,Dual Core 2.0 Ghz,Dual Core 2.0 Ghz,1 GB RAM,128mb Video,160 MB,2020
3,The Riftbreaker Heart of the Swamp,Your Price:106.00 SR,and operating system,Intel i5 gen 2 or AMD Bulldozer {4 cores),8 GB RAM,Nvidia GTX 750 2GB or AMD R7 265 2GB,12 GB,2024
4,Soulash,Your Price:62.00 SR,2 cores,2 cores,500 MB RAM,OpenGL,400 MB,2022


In [383]:
# dealing with the new nulls in Processor_main
# calculate the mode of the 'Processor' for each 'Memory' group
processor_mode_per_memory2 = df_steam_test.groupby('Memory')['Processor_main'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

# fill the missing 'Processor' values with the calculated mode
df_steam_test['Processor_main'].fillna(processor_mode_per_memory2, inplace=True)

In [387]:
#dropping the 4 nulls of processor because the processor is unique 
df_steam_test = df_steam_test.dropna(subset=['Processor_main'])


df_steam_test[df_steam_test['Processor_main'].isnull()]

,Game Names,Price,Processor_main,Memory,Graphics,Storage,years


In [388]:
df_steam_test.drop(['Processor'], axis=1, inplace=True)

KeyError: "['Processor'] not found in axis"

In [389]:
df_steam_test.to_csv('final_cleaned_data_steam2.csv')

In [395]:
df_steam_test

,Game Names,Price,Processor_main,Memory,Graphics,Storage,years
0,Kingdom Rush Origins Tower Defense,Your Price:248.95 SR,Dual Core CPU,1 GB RAM,OpenGL 3.0 compliant with 512MB of video RAM.,500 MB,2018
1,War Thunder Su39 Pack,No Price,"2,2 GHz",1 GB RAM,Nvidia GeForce 6XXX series or higher; AMD Rade...,2 GB,2023
2,The Battle of Polytopia,79.04 SR,Dual Core 2.0 Ghz,1 GB RAM,128mb Video,160 MB,2020
3,The Riftbreaker Heart of the Swamp,Your Price:106.00 SR,Intel i5 gen 2 or AMD Bulldozer {4 cores),8 GB RAM,Nvidia GTX 750 2GB or AMD R7 265 2GB,12 GB,2024
4,Soulash,Your Price:62.00 SR,2 cores,500 MB RAM,OpenGL,400 MB,2022
...,...,...,...,...,...,...,...
8519,Siege Survival Gloria Victis,118.80 SR,Intel Core i3 or AMD Athlon II X4 with at leas...,4 GB RAM,"GeForce GT 740, Radeon HD 7750 - at least 1GB ...",10 GB,2021
8520,Lay a Beauty to Rest A Butterfly Dream,34.10 SR,Intel Core i3 2.00 GHz 或 AMD A10,2 GB RAM,Intel® HD Graphics 3000,3 GB,2022
8521,Pamali Indonesian Folklore Horror,34.64 SR,Intel(R) Core(TM) i5-7200U CPU @ 2.50Ghz,4 GB RAM,NVIDIA GeForce 940MX,2 GB,2018
8522,A fascinating story,56.04 SR,Intel Core i5,4 GB RAM,NVIDIA Gef750,8 GB,2024
